In [3]:
from collections import defaultdict, namedtuple
from importlib import reload
from pathlib import Path
import sys
from string import ascii_lowercase as LETTERS

from sklearn.datasets import fetch_20newsgroups

sys.path.insert(0, str(Path.cwd().parents[0] / 'src')) 
import crypto as cg; reload(cg)

<module 'crypto' from '/Users/chris/Documents/cryptogram-solver/src/crypto.py'>

In [4]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=20180827)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=20180827)

In [5]:
reload(cg)
tokenizer = cg.Tokenizer(char_ngram_range=(1, 6), word_ngram_range=(1, 1), vocab_size=100000)
tokenizer.fit(news_train['data'][:500])
print(len(tokenizer.vocab))
print(tokenizer.totals)

100%|██████████| 500/500 [00:08<00:00, 58.49it/s]


100000
defaultdict(<class 'int'>, {(<NgramKind.word: 0>, 1): 150083, (<NgramKind.char: 1>, 1): 873873, (<NgramKind.char: 1>, 2): 739749, (<NgramKind.char: 1>, 3): 611312, (<NgramKind.char: 1>, 4): 492479, (<NgramKind.char: 1>, 5): 378646, (<NgramKind.char: 1>, 6): 276700})


In [6]:
doc2 = cg.Doc(news_test['data'][10])
# doc2 = prep.Doc(doc2.swap('a', 'b'))
# doc2.scramble(15)
doc2.text

'from: ez027993@dale.ucdavis.edu (gary built like villanueva huckabay)\nsubject: ron gant, stalling, and hirschbeck.\norganization: julio machado candlelight vigil society\nlines: 88\n\n(dale "seer" stephenson) writes:\n>while i can see why they want to cut down on the time spent walking around,\n>gant wasn\'t ordered back in the box "sooner", he was ordered back immediately.\n>as soon as he stepped out, hirschbeck told him to get back in the box.\n>now, gant doesn\'t take a lot of walks between pitches.  the only reason\n>he did then because he was *very* bothered by the call.  i expect his\n>concentration wasn\'t there yet, and in a crucial situation i imagine it\'s\n>best to be as calm as possible.  contributing factors would be gant\'s\n>bad day at the plate, bad year at the plate, and the braves long scoring\n>drought.\n\nand it is not hirschbeck\'s job to help gant with any of these difficulties.\nif gant can\'t gather his concentration for whatever reason, that just makes\nhim a

In [9]:
from random import sample
import numpy as np

In [15]:
class Solver:
    def __init__(self, tokenizer, doc):
        self.tokenizer = tokenizer
        self.doc = doc

    def calc_nll(self, text):
        """Caluclate mean negative log likelihood."""
        tokens = self.tokenizer.tokenize(text)
        nll = 0
        for token, count in tokens.items():
            try:
                vocab_count = self.tokenizer.vocab[token]
            except KeyError:
                vocab_count = self.tokenizer.pseudo_count
            total = self.tokenizer.totals[(token.kind, token.n)]  # could break
            prob = vocab_count / total
            nll += -1 * log(prob) * count
        return nll / len(tokens)  # take mean
    
    def softmax(x, temp=1):
        return np.exp(x / temp) / np.sum(np.exp(x / temp), axis=0)

In [21]:
Solver.softmax(np.array([2, 3, 4, 5]), 0.3)

array([  4.37804039e-05,   1.22723586e-03,   3.44014153e-02,
         9.64327568e-01])

In [22]:
from scipy.optimize import basinhopping

In [ ]:
basinhopping()

In [107]:
def solve_cryptogram(text, tokenizer, max_epochs=50, is_debug=True):
    mapping = Mapping()
    doc = cg.Doc(text)
    best_nll = tokenizer.calc_nll(doc.text)
    best_swap = ('a', 'a')
    epoch = 0
    improving = True
    while improving and epoch < max_epochs:
        improving = False
        print('*', end='')
        seen = set()
        doc_letters = get_letters(doc.translate())
        for l1 in doc_letters:
            for l2 in letters_set - set(l1):
                swap = tuple(sorted([l1, l2]))
                if swap in seen:
                    continue
                seen.add(swap)
                doc.swap(*swap)
                nll = tokenizer.calc_nll(doc.translate())
                if nll < best_nll:
                    best_swap = swap
                    best_nll = nll
                    improving = True
        mapping = cg.Doc(doc.swap(*best_swap))
        epoch += 1
        print(f'{doc.text}')
        print(f'best of epoch: {best_nll:0.5g}')
    print(f'\nfinal best ({epoch} epochs): {best_nll:0.5g}')
    doc.mapping = mapping
    return doc.translate()

In [110]:
text = doc2.text
text = 'Rbo rpktigo vcrb bwucja wj kloj hcjd, km sktpqo, cq rbwr loklgo vcgg cjqcqr kj skhcja wgkja wjd rpycja rk ltr rbcjaq cj cr. -- Roppy Lpwrsborr'
text = 'This is the story of a girl. Who cried a river and drowned the whole world. And while she looked so sad in photographs, I absolutely loved her when she smiled.'
doc = cg.Doc(text)
doc.scramble()
doc.translate()

TypeError: 'set' object does not support indexing

In [87]:
# text, options = solve_cryptogram(doc.text, tokenizer)
text = solve_cryptogram(doc.text, tokenizer)
print(text)

*

AttributeError: 'Doc' object has no attribute 'letters'

In [517]:
from collections import Counter

In [518]:
Counter(['a', 'b', 'c', 'c'])

Counter({'a': 1, 'b': 1, 'c': 2})

In [23]:
def softmax(options, temp=1):
    nums = [exp(x / temp) for x in options.values()]
    denom = sum(nums)
    choice_probs = [x / denom for x in nums]
    choice = random.choices(list(options.keys()), weights=choice_probs)[0]
    return choice


# options = {'a': 1, 'b': 2}
softmax(options, temp=0.05)

NameError: name 'options' is not defined

In [1775]:
softmax(options)

[4.5397868702434395e-05, 0.9999546021312976]


'b'